In [150]:
"""Docstring"""

import csv
import collections
import re

#Read Data
def readdata():
    # This function reads data from the original dataset
    with open('all_stocks_5yr.csv') as fp:
        reader = csv.reader(fp)
        data = list(reader)
    return data

def assetlist():
    # This function returns a set of tickers of S&P500 stocks
    n = readdata()
    k = [ i[6] for i in n]
    return set(k)

assetlist = assetlist() # A list of all stocks in the dataset for future reference
dataset = readdata() # The original dataset for future reference
data = dataset[1:] # The original dataset without header for future reference

#Format Data
def cleanrow(row):
    namepattern = r'[A-Z]*' # To check that all tickers contains only capital letters
    datepattern = r'(\d{4})-(\d{2})-(\d{2})*' # To check that the dates are in the YYYY-MM-DD format
    matchname = re.search(namepattern, row[6])
    matchdate = re.search(datepattern, row[0])
    
    new_row = [0, 0, 0, 0, 0, 0, 0]
    new_row[0] = row[0] if bool(matchdate) == True else None 
    new_row[1] = round(float(row[1]),2) if row[1] else None
    new_row[2] = round(float(row[2]),2) if row[2] else None
    new_row[3] = round(float(row[3]),2) if row[3] else None
    new_row[4] = round(float(row[4]),2) if row[4] else None
    new_row[5] = int(row[5]) if row[5].isdigit() else None
    new_row[6] = row[6] if bool(matchname) == True else None
    # format the original data, also check for missing data and some cases of wrong data
    # missing and wrong data are replaced by None so we could deal with them more efficiently
    # in the future
    
    return new_row

def cleandata(data):
    newdata = data
    for i in range(len(data)):
        newdata[i] = cleanrow(data[i])
    return newdata

cleandata = [dataset[0]] + cleandata(data) # Cleaned data with header

#Find possible wrong value


#Find missing value
#Replace wrong value
#Deal with missing value

In [153]:
class Equity:
    def __init__(self, name):
        self.name = name
        self.history = [dataset[0]]+[i for i in cleandata if i[6] == name]
    
    def RoR(self, day1, day2):
        s = [i[4] for i in self.history if i[0] == day1][0]
        St = [i[4] for i in self.history if i[0] == day2][0]
        return (St/s)-1

In [154]:
ASS = Equity('AAPL')
ASS.RoR('2013-02-28','2013-03-28')

0.0028544243577546258

In [1]:
import data_analysis

Missing Values
Missing values for: "date"
Empty DataFrame
Columns: [date, open, high, low, close, volume, Name]
Index: []


Missing values for: "open"
              date  open   high    low     close   volume  Name
82949   2017-07-26   NaN    NaN    NaN   69.0842        3   BHF
165734  2015-07-17   NaN  88.76  88.24   88.7200  2056819   DHR
165857  2016-01-12   NaN    NaN    NaN   88.5500        0   DHR
205076  2015-07-17   NaN  48.49  47.85   47.9200  1246786    ES
239832  2016-07-01   NaN    NaN    NaN   49.5400        0   FTV
434379  2015-07-17   NaN  47.31  46.83   46.9900  1229513     O
434502  2016-01-12   NaN    NaN    NaN   52.4300        0     O
478594  2015-06-09   NaN    NaN    NaN  526.0900    12135  REGN
558213  2016-04-07   NaN    NaN    NaN   41.5600        0    UA
581906  2015-05-12   NaN    NaN    NaN  124.0800   569747  VRTX
598236  2015-06-26   NaN    NaN    NaN   61.9000      100   WRK


Missing values for: "high"
              date  open  high  low     close  volum